## Generating velocity fields for acwv

`acwv` uses VTK file format, with ImageData structure, to store the velocity fields. This notebook shows how to generate these files.


In [1]:
import numpy as np
from matplotlib import pyplot as plt
plt.style.use("ggplot")

In [2]:
def modelo_planos_paralelos(vel, Nx, Nz):
    for i in range(Nx):
        for j in range(int(Nz/3)):
            vel[j][i] = 1500
        for j in range(int(Nz/3), int(2*Nz/3)):
            vel[j][i] = 2000
        for j in range(int(2*Nz/3), Nz):
            vel[j][i] = 3000
    return vel

def modelo_semicirculo(vel, Nx, Nz):
    for i in range(Nx):
        for j in range(Nz):
            raio = Nx/2
            circulo = -np.sqrt((raio**2)-(i - raio)**2) + Nz
            if j < Nz/3:
                vel[j][i] = 1500
            elif j < circulo and j >= Nz/3:
                vel[j][i] = 2000
            else:
                vel[j][i] = 3000
    return vel

def modelo_camada_dobrada(vel, Nx, Nz):
    raio = Nz/2;
    for i in range(Nx):
        if i < raio:
            area = np.sqrt(raio**2 - i**2) + Nz/2
        else:
            area = -np.sqrt(raio**2 - (i - Nx)**2) + Nz/2
        for j in range(Nz):
            if j < area:
                vel[j][i] = 2500
            else:
                vel[j][i] = 3500
    return vel

def modelo_reservatorio(vel, Nx, Nz):
    me = 70
    for i in range(Nx):
        for j in range(Nz):
            y = (8*me**3)/(i**2 + 4*me**2) ## regiao do reservatorio
            if j <= y:
                vel[j][i] = 1500
            elif j > y and j < me:
                vel[j][i] = 2000
            elif j >= me and j < y + 50 and j < 147:
                vel[j][i] = 2500
            elif j >= 147 and j < y + 50:
                vel[j][i] = 3000
            elif j >= y + 50 and j < 230:
                vel[j][i] = 3500
            else:
                vel[j][i] = 4000
    return vel

def visualizar_campos(vel):
    plt.figure(figsize=(7, 7))
    im = plt.imshow(vel, cmap='viridis')
    plt.colorbar(im)
    plt.grid(False)
    plt.show()

def slit(vel):
    mid_x = int(len(vel[0])/2)
    mid_y = int(len(vel)/2)
    thickness = 10
    for i in range(0, len(vel[0])):
        for j in range(0, len(vel)):
            if j in range(mid_y - thickness, mid_y + thickness) and \
               i not in range(mid_x - thickness, mid_x + thickness):
                vel[j][i] = 0
            else:
                vel[j][i] = 2000
                
def double_slit(vel):
    Nx = len(vel[0])
    Nz = len(vel)
    thickness = 10
    for i in range(0, Nx):
        for j in range(0, Nz):
            if j in range(int(Nz/2) - thickness, int(Nz/2) + thickness) and \
               (i not in range(int(Nx/3) - thickness, int(Nx/3) + thickness) and \
                i not in range(int(2*Nx/3) - thickness, int(2*Nx/3) + thickness)):
                vel[j][i] = 0
            else:
                vel[j][i] = 2000
                    
def salva_arquivo(nome_do_arquivo, Nx, Nz, Nt, dx, dt, vel):
    file = open(nome_do_arquivo, "w")
    file.write("{} {} {} {} {}\n".format(Nx, Nz, Nt, dx, dt))
    for i in range(Nx):
        for j in range(Nz):
            file.write("{} ".format(vel[j][i]))
    file.close()

def saveVtkImageDataXML(fname, Nx, Nz, dx, vel):
    file = open(fname, "w")
    file.write("<?xml version=\"1.0\"?>\n")
    file.write("<VTKFile type=\"ImageData\" version=\"0.1\" byte_order=\"LittleEndian\">\n")
    file.write("  <ImageData WholeExtent=\"0 {} 0 {} 0 {}\" Origin=\"0 0 0\" Spacing=\"{} {} {}\">\n".format(Nx-1, Nz-1, 0, dx, dx, 0))
    file.write("    <Piece Extent=\"0 {} 0 {} 0 {}\">\n".format(Nx-1, Nz-1, 0))
    file.write("      <PointData Scalars=\"velocity\">\n")
    file.write("        <DataArray type=\"Float32\" Name=\"velocity\" format=\"ascii\">\n")
    for i in range(Nx):
        for j in range(Nz):
            file.write("{} ".format(vel[j][i]))
    file.write("\n")
    file.write("        </DataArray>\n")
    file.write("      </PointData>\n")
    file.write("    </Piece>\n")
    file.write("  </ImageData>\n")
    file.write("</VTKFile>\n")
    file.close()

def saveVtkImageDataXMLBinary(fname, Nx, Nz, dx, vel):
    file = open(fname, "wb")
    file.write("<?xml version=\"1.0\"?>\n".encode())
    file.write("<VTKFile type=\"ImageData\" version=\"0.1\" byte_order=\"LittleEndian\">\n".encode())
    file.write("  <ImageData WholeExtent=\"0 {} 0 {} 0 {}\" Origin=\"0 0 0\" Spacing=\"{} {} {}\">\n".format(Nx-1, Nz-1, 0, dx, dx, 0).encode())
    file.write("    <Piece Extent=\"0 {} 0 {} 0 {}\">\n".format(Nx-1, Nz-1, 0).encode())
    file.write("      <PointData Scalars=\"velocity\">\n".encode())
    file.write("        <DataArray type=\"Float32\" Name=\"velocity\" format=\"binary\">\n".encode())
    vel.tofile(file)
    file.write("\n".encode())
    file.write("        </DataArray>\n".encode())
    file.write("      </PointData>\n".encode())
    file.write("    </Piece>\n".encode())
    file.write("  </ImageData>\n".encode())
    file.write("</VTKFile>\n".encode())
    file.close()


In [4]:
Nx = 300
Nz = 300
dx = 10
vel = np.zeros((Nz, Nx), float)
modelo_planos_paralelos(vel, Nx, Nz)
saveVtkImageDataXML("../data/input.vti", Nx, Nz, dx, vel)